In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import os

In [3]:
import tensorflow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
from keras.models import Model,Sequential

In [4]:
from numpy.linalg import norm
import os
from tqdm import tqdm
import pickle

In [5]:
model = ResNet50(weights='imagenet',include_top = False,input_shape = (244,244,3))
model.trainable = False

2022-10-25 05:02:53.131720: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-25 05:02:53.132719: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-25 05:02:53.286144: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-25 05:02:53.287001: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-25 05:02:53.287759: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

94781440/94765736 [==============================] - 0s 0us/step


In [6]:
model = Sequential([model,GlobalMaxPooling2D()])

In [7]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 8, 8, 2048)        23587712  
_________________________________________________________________
global_max_pooling2d (Global (None, 2048)              0         
Total params: 23,587,712
Trainable params: 0
Non-trainable params: 23,587,712
_________________________________________________________________
None


In [8]:
# def extract_features(image_path,model):
#     img = image.load_img(img_path,target_size=(224,224))
#     img_array = image.img_to_array(img)
#     expanded_img_array = np.expand_dims(img_array, axis=0)
#     preprocessed_img = preprocess_input(expanded_img_array)
#     result = model.predict(preprocessed_img).flatten()
#     normalized_result = result / norm(result)
    
#     return normalized_result

In [9]:
def extract_features(file,model):
    img = image.load_img(file,target_size=(244,244))
    img_array = image.img_to_array(img)
#     img = np.reshape(img,[none,240,240,3])
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    result = model.predict(preprocessed_img).flatten()
    normalized_result = result / norm(result)
    
    return normalized_result

In [10]:
filenames = []

In [11]:
for file in os.listdir('../input/fashion-product-images-small/images'):
    filenames.append(os.path.join('../input/fashion-product-images-small/images',file))

In [12]:
print(len(filenames))

44441


In [13]:
print(filenames[0:5])

['../input/fashion-product-images-small/images/31973.jpg', '../input/fashion-product-images-small/images/30778.jpg', '../input/fashion-product-images-small/images/19812.jpg', '../input/fashion-product-images-small/images/22735.jpg', '../input/fashion-product-images-small/images/38246.jpg']


In [14]:
feature_list = []

In [15]:
for file in tqdm(filenames):
    feature_list.append(extract_features(file,model))

  0%|          | 0/44441 [00:00<?, ?it/s]2022-10-25 05:03:00.414171: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-10-25 05:03:02.621964: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005
100%|██████████| 44441/44441 [48:00<00:00, 15.43it/s]  


In [17]:
pickle.dump(feature_list,open('featureslist.pkl','wb'))
pickle.dump(filenames,open('filenames.pkl','wb'))